# Tensorflow - Neural Networkds

We've discovered that the data may need more complex modeling, so we can test neural networks

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk

## Model 1: Simple Linear Model

This model will feature no hidden layers, and will simply use weights (mulitplying inputs) and bias (adding to the result) in order to create a one vs all probability of each class. For model evalution, we will use a test-train split.

Note: We will treat each seperate class of polviews as it's own class.

### Data Prep

With a few modifications, we can use the output of the data pipeline.

In [2]:
X = pd.read_csv('data/training/X.csv')
y = pd.read_csv('data/training/y.csv', names=['Polviews'])

In [3]:
from sklearn.preprocessing import OneHotEncoder
y = OneHotEncoder(categories='auto').fit_transform(y).todense()

In [4]:
# Test/Train Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [65]:
# Number of X columns
X_cols = X.shape[1]

# Number of y columns
y_cols = y.shape[1]

# List of the true value for each column
y_true = np.squeeze(np.asarray(y.argmax(axis=1)))
y_train_true = np.squeeze(np.asarray(y_train.argmax(axis=1)))
y_test_true = np.squeeze(np.asarray(y_test.argmax(axis=1)))

### Placeholder Variables

These variables are placeholders for our Tensorflow graph.

In [66]:
x = tf.placeholder(tf.float32, [None, X_cols])
y_true = tf.placeholder(tf.float32, [None, y_cols])
y_true_cls = tf.placeholder(tf.int64, [None])

In [67]:
weights = tf.Variable(tf.zeros([X_cols, y_cols]))
biases = tf.Variable(tf.zeros([y_cols]))

In [68]:
logits = tf.matmul(x, weights) + biases
y_pred = tf.nn.softmax(logits)
y_pred_cls = tf.argmax(y_pred, axis=1)

### Cost function, Optimization, and Accuarcy

In [69]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                           labels=y_true)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### Running + Helper funcions

In [70]:
session = tf.Session()
session.run(tf.global_variables_initializer())
batch_size = 100

In [71]:
def optimize(num_iterations):
    for i in range(num_iterations):
        # Get a batch of training examples.
        # x_batch now holds a batch of images and
        # y_true_batch are the true labels for those images.
        x_batchjn  atch = tf.train.shuffle_batch(
      [X_train, y_train],
      batch_size=500
      num_threads=4,
      capacity=50000,
      min_after_dequeue=10000)
        
        # Put the batch into a dict with the proper names
        # for placeholder variables in the TensorFlow graph.
        # Note that the placeholder for y_true_cls is not set
        # because it is not used during training.
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}

        # Run the optimizer using this batch of training data.
        # TensorFlow assigns the variables in feed_dict_train
        # to the placeholder variables and then runs the optimizer.
        session.run(optimizer, feed_dict=feed_dict_train)

In [77]:
image_batch, label_batch = tf.train.shuffle_batch(
      [X_train, y_train],
      batch_size=32,
      num_threads=4,
      capacity=50000,
      min_after_dequeue=10000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


In [72]:
feed_dict_test = {x: X_test,
                  y_true: y_test,
                  y_true_cls: y_test_true}

In [73]:
def print_accuracy():
    # Use TensorFlow to compute the accuracy.
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    # Print the accuracy.
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [74]:
def print_confusion_matrix():
    # Get the true classifications for the test-set.
    cls_true = y_trueest
    
    # Get the predicted classifications for the test-set.
    cls_pred = session.run(y_pred_cls, feed_dict=feed_dict_test)

    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    # Print the confusion matrix as text.
    print(cm)

    # Plot the confusion matrix as an image.
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)

    # Make various adjustments to the plot.
    plt.tight_layout()
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [75]:
# No iterations
print_accuracy()

Accuracy on test-set: 3.6%


In [76]:
optimize(num_iterations=500)

NameError: name 'data' is not defined

In [ ]:
session.close()